In [0]:
!nvidia-smi

Sun May  3 15:50:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
a = []
while(1):
    a.append('1')

In [1]:
!git clone https://github.com/openai/jukebox.git

Cloning into 'jukebox'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 615 (delta 2), reused 1 (delta 0), pack-reused 606
Receiving objects: 100% (615/615), 2.70 MiB | 2.46 MiB/s, done.
Resolving deltas: 100% (211/211), done.


In [2]:
%cd /content/jukebox


/content/jukebox


In [3]:
!pip install -r requirements.txt
!pip install -e .

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 245kB 7.5MB/s 
     |████████████████████████████████| 1.6MB 8.3MB/s 
     |████████████████████████████████| 1.4MB 11.1MB/s 
  Created wheel for fire: filename=fire-0.1.3-py2.py3-none-any.whl size=49704 sha256=a36fe174a456190e1559937a97e07dc66da2b6bf8697f0504cf153f1a4056563
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Created wheel for librosa: filename=librosa-0.7.2-cp36-none-any.whl size=1612885 sha256=bb47af5763601e28d4576c34bd1827925178eec06e6365a1ce1cdae94c93f7b2
  Stored in directory: /root/.cache/pip/wheels/4c/6e/d7/bb93911540d2d1e44d690a1561871e5b6af82b69e80938abef
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp36-cp36m-linux_x86_64.whl size=2074410 sha256=ac361d93414a6740404361268f60720501755c720b5cd4fa4a00549245085f60
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59

Obtaining file:///content/jukebox
  Running setup.py develop for jukebox


In [0]:
import jukebox
import torch as t
import librosa
import os
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()

Using cuda True


In [0]:
model = "1b_lyrics" # or "5b_lyrics"     
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = 3 if model=='5b_lyrics' else 8
hps.name = 'samples'
chunk_size = 16 if model=="5b_lyrics" else 32
max_batch_size = 3 if model=="5b_lyrics" else 16
hps.levels = 3
hps.hop_fraction = [.5,.5,.125]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)



Restored from /root/.cache/jukebox-assets/models/5b/vqvae.pth.tar
0: Loading vqvae in eval mode
Creating cond. autoregress with prior bins [79, 2048], 
dims [384, 6144], 
shift [ 0 79]
input shape 6528
input bins 2127
Self copy is False
Loading artist IDs from /content/jukebox/jukebox/data/ids/v3_artist_ids.txt
Loading artist IDs from /content/jukebox/jukebox/data/ids/v3_genre_ids.txt
Level:2, Cond downsample:None, Raw to tokens:128, Sample length:786432
Restored from /root/.cache/jukebox-assets/models/1b_lyrics/prior_level_2.pth.tar
0: Loading prior in eval mode


In [0]:
sample_length_in_seconds = 20          

hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

In [0]:
metas = [dict(artist = "Zac Brown Band",
            genre = "Rock",
            total_length = hps.sample_length,
            offset = 0,
            lyrics = """I met a traveller from an antique land,
            Who said—“Two vast and trunkless legs of stone
            Stand in the desert. . . . Near them, on the sand,
            Half sunk a shattered visage lies, whose frown,
            And wrinkled lip, and sneer of cold command,
            Tell that its sculptor well those passions read
            Which yet survive, stamped on these lifeless things,
            The hand that mocked them, and the heart that fed;
            And on the pedestal, these words appear:
            My name is Ozymandias, King of Kings;
            Look on my Works, ye Mighty, and despair!
            Nothing beside remains. Round the decay
            Of that colossal Wreck, boundless and bare
            The lone and level sands stretch far away
            """,
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]

In [0]:
sampling_temperature = .98

lower_batch_size = 16
max_batch_size = 3 if model == "5b_lyrics" else 16
lower_level_chunk_size = 32
chunk_size = 16 if model == "5b_lyrics" else 32
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True, 
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

In [0]:
zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(len(priors))]
zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)

Sampling level 2
Sampling 6144 tokens for [0,6144]. Conditioning on 0 tokens
Ancestral sampling 3 samples with temp=0.98, top_k=0, top_p=0.0
12/12 [00:06<00:00,  1.77it/s]
6144/6144 [11:53<00:00,  8.61it/s]
Sampling 6144 tokens for [746,6890]. Conditioning on 5398 tokens
Primed sampling 3 samples with temp=0.98, top_k=0, top_p=0.0
181/181 [01:59<00:00,  1.51it/s]
746/746 [01:32<00:00,  8.03it/s]


In [0]:
Audio(f'{hps.name}/level_2/item_0.wav')

In [0]:
# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization during the upsampling stage). For a hosted runtime, 
# we'll need to go ahead and delete the top_prior if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

Conditioning on 1 above level(s)
Checkpointing convs
Checkpointing convs
Loading artist IDs from /content/jukebox/jukebox/data/ids/v2_artist_ids.txt
Loading artist IDs from /content/jukebox/jukebox/data/ids/v2_genre_ids.txt
Level:0, Cond downsample:4, Raw to tokens:8, Sample length:65536
Restored from /root/.cache/jukebox-assets/models/5b/prior_level_0.pth.tar
0: Loading prior in eval mode
Conditioning on 1 above level(s)
Checkpointing convs
Checkpointing convs
Loading artist IDs from /content/jukebox/jukebox/data/ids/v2_artist_ids.txt
Loading artist IDs from /content/jukebox/jukebox/data/ids/v2_genre_ids.txt
Level:1, Cond downsample:4, Raw to tokens:32, Sample length:262144
Restored from /root/.cache/jukebox-assets/models/5b/prior_level_1.pth.tar
0: Loading prior in eval mode


In [0]:
zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)


Sampling level 1
Sampling 8192 tokens for [0,8192]. Conditioning on 0 tokens
Ancestral sampling 3 samples with temp=0.99, top_k=0, top_p=0.0
8192/8192 [14:41<00:00,  9.29it/s]
Sampling 8192 tokens for [4096,12288]. Conditioning on 4096 tokens
Primed sampling 3 samples with temp=0.99, top_k=0, top_p=0.0
128/128 [00:49<00:00,  2.59it/s]
4096/4096 [07:30<00:00,  9.09it/s]
Sampling 8192 tokens for [8192,16384]. Conditioning on 4096 tokens
Primed sampling 3 samples with temp=0.99, top_k=0, top_p=0.0
128/128 [00:49<00:00,  2.59it/s]
4096/4096 [07:29<00:00,  9.12it/s]
Sampling 8192 tokens for [12288,20480]. Conditioning on 4096 tokens
Primed sampling 3 samples with temp=0.99, top_k=0, top_p=0.0
128/128 [00:49<00:00,  2.59it/s]
4096/4096 [07:28<00:00,  9.13it/s]
Sampling 8192 tokens for [16384,24576]. Conditioning on 4096 tokens
Primed sampling 3 samples with temp=0.99, top_k=0, top_p=0.0
128/128 [00:49<00:00,  2.59it/s]
4096/4096 [07:28<00:00,  9.13it/s]
Sampling 8192 tokens for [19368,27560]

In [0]:
Audio(f'{hps.name}/level_0/item_0.wav')